# Mechanisms of Action Prediction

<br>

### Contents

* <a href="#intro">1. Introduction</a>
   * <a href="#datadec">1.1. Data description</a>
- <a href="#data">2. Data overview and data preprocessing</a>
* <a href="#eda">3. Exploratory Data Analysis</a>
   * <a href="#cp">3.1. Treatment features (cp_type, cp_time, cp_dose)</a>
   * <a href="#gene">3.2. Gene expression features</a>
   - <a href="#cell">3.3. Cell viability features</a>
   - <a href="#corr">3.4. Correlations: cell viability and gene expression features</a>
   - <a href="#targets">3.5. MoA Targets</a>
- <a href="#pca">4. Dimensionality reduction via PCA</a>
   - <a href="#pcag">4.1. PCA: Gene expression</a>
   - <a href="#pcac">4.2. PCA: Cell viability</a>
   - <a href="#pcac">4.3. PCA: Gene expression and Cell viability</a>
- <a href="#lr">5. A Logistic Regression Model (neural networks without hidden layers)</a>
   - <a href="#data-prep">5.1. Data preprocessing</a>
   - <a href="#hyper-lr">5.2. Lambda optimization</a>
   - <a href="#train-original-lr">5.3. Predictions for the original data</a>
   - <a href="#train-red-lr">5.4. Predictions for the reduced data</a>
   - <a href="#train-pca-lr">5.5. Predictions for the PCA data</a>
   - <a href="#train-pca_wv-lr">5.6. Predictions for the reduced PCA data
   - <a href="#comp-lr">5.7. Comparing the results</a>
- <a href="#dnn">6. Multi-label classification via Deep Neural Networks</a>
   - <a href="#hyper">6.1. Hyperparameter optimization</a>
   - <a href="#train-original">6.2. Predictions for the original data</a>
   - <a href="#train-red">6.3. Predictions for the reduced data</a>
   - <a href="#train-pca">6.4. Predictions for the PCA data</a>
   - <a href="#train-pca_wv">6.5. Predictions for the reduced PCA data
   - <a href="#comp">6.6. Comparing the results</a>
   - <a href="#comp_models">6.7. Comparing the models</a>
- <a href="#submit">7. Submission</a>

<div id="intro"></div>

## 1. Introduction

The [Connectivity Map](https://clue.io/), a project within the Broad Institute of MIT and [Harvard, the Laboratory for Innovation Science at Harvard (LISH)](https://lish.harvard.edu/), and the NIH Common Funds Library of Integrated Network-Based Cellular Signatures (LINCS), present this challenge with the goal of advancing drug development through improvements to MoA prediction algorithms.


<b>What is the Mechanism of Action (MoA) of a drug? And why is it important?</b>

In the past, scientists derived drugs from natural products or were inspired by traditional remedies. Very common drugs, such as paracetamol, known in the US as acetaminophen, were put into clinical use decades before the biological mechanisms driving their pharmacological activities were understood. Today, with the advent of more powerful technologies, drug discovery has changed from the serendipitous approaches of the past to a more targeted model based on an understanding of the underlying biological mechanism of a disease. In this new framework, scientists seek to identify a protein target associated with a disease and develop a molecule that can modulate that protein target. As a shorthand to describe the biological activity of a given molecule, scientists assign a label referred to as mechanism-of-action or MoA for short.

<br>

<div id="datadec"></div>

### 1.1. Data description

<code>train_features.csv</code>: Features for the training set. Features g- signify gene expression data, and c- signify cell viability data. cp_type indicates samples treated with a compound (cp_vehicle) or with a control perturbation (ctrl_vehicle); control perturbations have no MoAs; cp_time and cp_dose indicate treatment duration (24, 48, 72 hours) and dose (high or low).

<code>cp_type (categorical)</code>: Samples treated with a compound or with a control perturbation. Categories include "trt_cp" and "ctl_vehicle", respectively. There is no MoA for "ctl_vehicle".

<code>cp_time (categorical)</code>: Treatment duration in hours. Categories include 24, 48, and 72 hours.

<code>cp_dose (categorical)</code>: Drug dose. Categories include "D1", "D2" for low and high dose, respectively.

<code>g-[0-771] (continous)</code>: Gene expression data - a measure of activation in a given gene after the drug is applied.

<code>c-[0-99] (continous)</code>: Cell viability. Basically count of live cells after the drug is applied.
    

<code>train_targets_scored.csv</code>: The binary MoA targets that are scored. There are 206 MoA targets.

<code>test_features.csv</code>: Features for the test data. We must predict the probability of each scored MoA for each row in the test data.
    

<hr>

<div id="data"></div>

## 2. Data overview and data preprocessing

In [ ]:
#Loading libraries / methods:

import warnings
warnings.filterwarnings("ignore")

import os
import time
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import CCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV
from sklearn.metrics import log_loss, classification_report, confusion_matrix
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Input
from keras.wrappers.scikit_learn import KerasClassifier
from keras import regularizers
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow_addons.layers import WeightNormalization
from tensorflow_addons.optimizers import Lookahead
from tqdm.keras import TqdmCallback

sns.set()
%matplotlib inline

In [ ]:
#Loading datasets (we'll ignore the non-scored dataset):

train_features = pd.read_csv('/kaggle/input/lish-moa/train_features.csv')
train_targets = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv')
test_features = pd.read_csv('/kaggle/input/lish-moa/test_features.csv')

In [ ]:
#Visualization of the train data:
train_features.head()

In [ ]:
#Visualization of the test data:
test_features.head()

In [ ]:
#Visualization of the target data:
train_targets.head()

In [ ]:
#Dimension of the datasets:

print('Number of rows in training set: ', train_features.shape[0])
print('Number of columns in training set: ', train_features.shape[1])
print('Number of rows in test set: ', test_features.shape[0])
print('Number of columns in test set: ', test_features.shape[1])
print('Number of rows in target set: ',train_targets.shape[0])
print('Number of columns in target set: ',train_targets.shape[1])

#The dataset is split into the following setting: 85% into the training 
#set (23814), 15% into the test set (3982).

In [ ]:
#There are 772 gene expression features (g-0 to g-771):
train_features.iloc[:, 4:776].head()

In [ ]:

#There are 100 cell viability features (c-0 to c-99):
train_features.iloc[:, 776:876].head()

In [ ]:
#Gene expression:
train_gene = pd.DataFrame(train_features.iloc[:, 4:776])

#Cell Viability:
train_cell = pd.DataFrame(train_features.iloc[:, 776:876])

#Gene expression and Cell Viability:
train_gc = pd.DataFrame(train_features.iloc[:, 4:876])
train_gc.head(5)

In [ ]:
#Data preprocessing (test_features)

#Converting cp_type and cp_dose as binary:
def Preprocess(data):
    data = data.copy()
    data.loc[:, 'cp_type'] = data.loc[:, 'cp_type'].map({'trt_cp': 0, 'ctl_vehicle': 1})
    data.loc[:, 'cp_dose'] = data.loc[:, 'cp_dose'].map({'D1': 0, 'D2': 1})
    del data['sig_id']
    return data

test = Preprocess(test_features)

In [ ]:
#Loading csv files
x_train_orig = pd.read_csv('/kaggle/input/orig-data/orig_data/x_train_orig.csv')
y_train_orig = pd.read_csv('/kaggle/input/orig-data/orig_data/y_train_orig.csv')
x_test_orig = pd.read_csv('/kaggle/input/orig-data/orig_data/x_test_orig.csv')
y_test_orig = pd.read_csv('/kaggle/input/orig-data/orig_data/y_test_orig.csv')

x_train_orig = np.asmatrix(x_train_orig)
y_train_orig = np.asmatrix(y_train_orig)
x_test_orig = np.asmatrix(x_test_orig)
y_tes_orig = np.asmatrix(y_test_orig)

print(x_train_orig.shape, y_train_orig.shape, x_test_orig.shape, y_test_orig.shape)

In [ ]:
#Loading csv files
x_train_wv = pd.read_csv('/kaggle/input/reddata/red_data/x_train_wv.csv')
y_train_wv = pd.read_csv('/kaggle/input/reddata/red_data/y_train_wv.csv')
x_test_wv = pd.read_csv('/kaggle/input/reddata/red_data/x_test_wv.csv')
y_test_wv = pd.read_csv('/kaggle/input/reddata/red_data/y_test_wv.csv')

x_train_wv = np.asmatrix(x_train_wv)
y_train_wv = np.asmatrix(y_train_wv)
x_test_wv = np.asmatrix(x_test_wv)
y_test_wv= np.asmatrix(y_test_wv)

print(x_train_wv.shape, y_train_wv.shape, x_test_wv.shape, y_test_wv.shape)

In [ ]:
#Loading csv files
x_train_pca = pd.read_csv('/kaggle/input/pca-data/pca_data/x_train_pca.csv')
y_train_pca = pd.read_csv('/kaggle/input/pca-data/pca_data/y_train_pca.csv')
x_test_pca = pd.read_csv('/kaggle/input/pca-data/pca_data/x_test_pca.csv')
y_test_pca = pd.read_csv('/kaggle/input/pca-data/pca_data/y_test_pca.csv')

x_train_pca = np.asmatrix(x_train_pca)
y_train_pca = np.asmatrix(y_train_pca)
x_test_pca = np.asmatrix(x_test_pca)
y_test_pca = np.asmatrix(y_test_pca)

print(x_train_pca.shape, y_train_pca.shape, x_test_pca.shape, y_test_pca.shape)

In [ ]:
#Loading csv files
x_train_pca_wv = pd.read_csv('/kaggle/input/pca-wv-data/pca_wv_data/x_train_pca_wv.csv')
y_train_pca_wv = pd.read_csv('/kaggle/input/pca-wv-data/pca_wv_data/y_train_pca_wv.csv')
x_test_pca_wv = pd.read_csv('/kaggle/input/pca-wv-data/pca_wv_data/x_test_pca_wv.csv')
y_test_pca_wv = pd.read_csv('/kaggle/input/pca-wv-data/pca_wv_data/y_test_pca_wv.csv')

x_train_pca_wv = np.asmatrix(x_train_pca_wv)
y_train_pca_wv = np.asmatrix(y_train_pca_wv)
x_test_pca_wv = np.asmatrix(x_test_pca_wv)
y_tes_pca_wv = np.asmatrix(y_test_pca_wv)

print(x_train_pca_wv.shape, y_train_pca_wv.shape, x_test_pca_wv.shape, y_test_pca_wv.shape)

# <div id="eda"></div>

## 3. Exploratory Data Analysis

<div id="cp"></div>

### 3.1. Treatment features (cp_type, cp_time, cp_dose)

In [ ]:
#Distribution of data in the training dataset, taking into consideration the
#features: cp_type, cp_time and cp_dose.

fig, axarr = plt.subplots(2, 2, figsize=(12, 12))

#cp_time
sns.countplot(train_features.select_dtypes('int64')['cp_time'], ax=axarr[0][0])
axarr[0][0].set_title("Exposure time (in hours)")

#cp_type
sns.countplot(train_features.select_dtypes('object')['cp_type'], ax=axarr[0][1])
axarr[0][1].set_title("Type of perturbation (compound or control)")

#cp_dose
sns.countplot(train_features.select_dtypes('object')['cp_dose'], ax=axarr[1][0])
axarr[1][0].set_title("Dosage (high or low)")

axarr[1][1].set_axis_off()

In [ ]:
#Distribution of data in the test dataset, taking into consideration the
#features: cp_type, cp_time and cp_dose.

fig, axarr = plt.subplots(2, 2, figsize=(12, 12))

#cp_time
sns.countplot(test_features.select_dtypes('int64')['cp_time'], ax=axarr[0][0])
axarr[0][0].set_title("Exposure time (in hours)")

#cp_type
sns.countplot(test_features.select_dtypes('object')['cp_type'], ax=axarr[0][1])
axarr[0][1].set_title("Type of perturbation (compound or vehicle)")

#cp_dose
sns.countplot(test_features.select_dtypes('object')['cp_dose'], ax=axarr[1][0])
axarr[1][0].set_title("Dosage (high or low)")

axarr[1][1].set_axis_off()

<div id="gene"></div>

### 3.2. Gene expression features

In [ ]:
#Distributions for the first five gene expression features
#with respect to the cp_time variable 

gene_columns = [cols for cols in train_features.columns if cols.startswith('g-')]

sns.pairplot(data=train_features[gene_columns[0:5]+['cp_time']],
             hue='cp_time', dropna=True, palette='Set2')

In [ ]:
#Distributions for the first five gene expression features
#with respect to the cp_dose variable 

sns.pairplot(data=train_features[gene_columns[5:10]+['cp_dose']],
             hue='cp_dose', dropna=True, palette='Set2')

In [ ]:
#Distributions for the first five gene expression features
#with respect to the cp_type variable 

sns.pairplot(data=train_features[gene_columns[10:15]+['cp_type']],
             hue='cp_type', dropna=True, palette='Set2')

<div id="cell"></div>

### 3.3. Cell viability features

In [ ]:
#Distributions for the first five cell viability features
#with respect to the cp_time variable 

cell_columns = [cols for cols in train_features.columns if cols.startswith('c-')]

sns.pairplot(data=train_features[cell_columns[0:5]+['cp_time']],
             hue='cp_time', dropna=True, palette='Set2')

In [ ]:
#Distributions for the first five cell viability features
#with respect to the cp_dose variable 

sns.pairplot(data=train_features[cell_columns[5:10]+['cp_dose']],
             hue='cp_dose', dropna=True, palette='Set2')

In [ ]:
#Distributions for the first five cell viability features
#with respect to the cp_type variable 

sns.pairplot(data=train_features[cell_columns[10:15]+['cp_type']],
             hue='cp_type', dropna=True, palette='Set2')

<div id="corr"></div>

### 3.4. Correlations: cell viability and gene expression features

In [ ]:
#Heatmap: Correlation matrices of 15 cell viability features and 15 gene expression features

frames = [train_gene.iloc[:, 1:30], train_cell.iloc[:, 1:30]]
df = pd.concat(frames)

corr = df.corr(method='pearson')

# plot
ax = plt.figure(figsize=(9,9))

cmap = sns.diverging_palette(220, 10, as_cmap=True)

sns.heatmap(corr, cmap=cmap, square=True, linewidths=.8, center = 0.8,
            cbar_kws={"label": "pearson correlation coefficient", "shrink": .6})

plt.title('Gene Expression and Cellular Viability Correlations', size=20, pad=10)

In [ ]:
#Canonical Correlation Analysis (CCA): Gene Exp. vs Cell Viability

cca = CCA(n_components=1)
cca.fit(train_gene, train_cell)

U_c, V_c = cca.transform(train_gene, train_cell)
plt.plot(U_c, V_c)
result = np.corrcoef(U_c.T, V_c.T)[0,1]
result

In [ ]:
#There is a linear relationship between the gene expression features and the cell viability features.

<div id="targets"></div>

### 3.5. MoA Targets

In [ ]:
#Distribution of MoA targets

moa_counts = train_targets.sum(axis=1)
ax = sns.countplot(moa_counts, palette='Set2')
ax.set_title("# MoAs by perturbation")

In [ ]:
#Frequency of MoA targets

for freq in set(moa_counts):
    frequency = len(moa_counts[moa_counts == freq])/len(moa_counts)
    
    print(str(freq)+': '+str(round(frequency,6)*100)+'%')

In [ ]:
#Most frequent MoA vs least frequent MoA target

per_moa_counts = train_targets.transpose()[1:].sum(axis=1).sort_values(ascending=False)

fig, axarr = plt.subplots(2, 2, figsize=(12, 12))

#more_MoAs
per_moa_counts[:5].sort_values(ascending=True).plot(kind='barh', x='index', legend=False, ax=axarr[0][0], color='orange')
axarr[0][0].set_title("Most frequent MoAs")

axarr[0][1].set_axis_off()

axarr[1][0].set_axis_off()

#less_MoAs
per_moa_counts[-5:].sort_values(ascending=True).plot(kind='barh', x='index', legend=False, ax=axarr[1][1], color='purple')
axarr[1][1].set_title("Least frequent MoAs")

<div id="pca"></div>

## 4. Dimensionality reduction via PCA

<div id="pcag"></div>

### 4.1. PCA: Gene expression

In [ ]:
#PCA: gene expression features
#n_components = 0.93: at least 93% of the data variance explained.

pca = PCA(n_components = 0.93)
g_pca = pca.fit_transform(train_features.iloc[:, 4:776]) 
g_pca.shape

In [ ]:
#The 442 Principal Components

pc_df = pd.DataFrame(data = g_pca, columns = range(1, 443))
pc_df = pc_df.add_prefix('PC')

pc_df['cp_type']= train_features['cp_type']
pc_df['cp_dose']= train_features['cp_dose']
pc_df['cp_time']= train_features['cp_time']

pc_df.head(3)

In [ ]:
#Scree plot: gene expression (explained variance ratio)

explained_variance_v = (pca.explained_variance_ratio_)*100 

per_var = np.round(explained_variance_v[:10,], decimals=1)
labels = ['PC' + str(x) for x in range(1, len(per_var)+1)]
 
plt.bar(x=range(1,len(per_var)+1), height=per_var, tick_label=labels)
plt.ylabel('Percentage of Explained Variance')
plt.xlabel('Principal Component')
plt.title('Scree Plot (gene expression)')
plt.show()

In [ ]:
#Cumulative explained variance for gene expression features

plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')
plt.title('PCA: Cumulative Explained Variance for Gene Expression Variables')

In [ ]:
#Distribution of the gene features w.r.t. the CP1 and CP2,
#highlighting the cp_time, cp_dose, and cp_type variables:

fig, (ax1, ax2, ax3) = plt.subplots(1,3,figsize=(20,5))

sns.scatterplot(x= "PC1", y= "PC2", data=pc_df, hue="cp_time", s=100, ax = ax1)
sns.scatterplot(x="PC1", y="PC2", data=pc_df, hue="cp_dose", s=100, ax = ax2, palette = "pastel")
sns.scatterplot(x="PC1", y="PC2", data=pc_df, hue="cp_type", s=100, ax = ax3, palette = "pastel")

<div id="pcac"></div>

### 4.2. PCA: Cell viability

In [ ]:
#PCA: cell viability features
#n_components = 0.93: at least 93% of the data variance explained.

pca2 = PCA(n_components = 0.93)
v_pca = pca2.fit_transform(train_features.iloc[:, 777:876])
v_pca.shape

In [ ]:
#The 30 Principal Components

viab_df = pd.DataFrame(data = v_pca, columns = range(1, 31))
viab_df = viab_df.add_prefix('PC')

viab_df['cp_type']= train_features['cp_type']
viab_df['cp_dose']= train_features['cp_dose']
viab_df['cp_time']= train_features['cp_time']

viab_df.head(3)

In [ ]:
#Scree plot: Cell viability (explained variance ratio)

explained_variance_v = (pca2.explained_variance_ratio_)*100 

per_var = np.round(explained_variance_v[:10,], decimals=1)
labels = ['PC' + str(x) for x in range(1, len(per_var)+1)]
 
plt.bar(x=range(1,len(per_var)+1), height=per_var, tick_label=labels)
plt.ylabel('Percentage of Explained Variance')
plt.xlabel('Principal Component')
plt.title('Scree Plot (cell viability)')
plt.show()

In [ ]:
#Cumulative explained variance for cellular viability features

plt.plot(np.cumsum(pca2.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')
plt.title('PCA: Cumulative Explained Variance for Cellular Viability Variables')

In [ ]:
#Distribution of the cell viab. features w.r.t. the CP1 and CP2,
#highlighting the cp_time, cp_dose, and cp_type variables:

fig2, (ax1, ax2, ax3) = plt.subplots(1,3,figsize=(20,5))

sns.scatterplot(x = "PC1", y = "PC2", data = viab_df, hue="cp_time", s = 100, ax = ax1)
sns.scatterplot(x = "PC1", y = "PC2", data = viab_df, hue="cp_dose", s = 100, ax = ax2, palette = "pastel")
sns.scatterplot(x = "PC1", y = "PC2", data = viab_df, hue="cp_type", s = 100, ax = ax3, palette = "pastel")

<div id="pca-gc"></div>

### 4.3. PCA: Gene expression and Cell viability

In [ ]:
#Standardization
scaler=StandardScaler()
scaler.fit(train_gc)

scaled_data=scaler.transform(train_gc)


#PCA: Gene expression & Cell vialbility
#n_components=0.95: Keep features explaining at least 95% of the variability.

pca_gc = PCA(n_components=0.95) #at least 95% of the variability.
pca_gc.fit(scaled_data)

gc_pca = pca_gc.transform(scaled_data)
print(scaled_data.shape, gc_pca.shape)

In [ ]:
#Cumulative explained variance for Gene Exp. and Cell Viab. features

plt.plot(np.cumsum(pca_gc.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')
plt.title('PCA: Cumulative Explained Variance for Gene Exp. and Cell Viab. Variables')

In [ ]:
print(np.cumsum(pca_gc.explained_variance_ratio_)[487])
print(np.cumsum(pca_gc.explained_variance_ratio_)[493])
print(np.cumsum(pca_gc.explained_variance_ratio_)[500])
print(np.cumsum(pca_gc.explained_variance_ratio_)[510])
print(np.cumsum(pca_gc.explained_variance_ratio_)[520])
print(np.cumsum(pca_gc.explained_variance_ratio_)[530])
print(np.cumsum(pca_gc.explained_variance_ratio_)[540])

In [ ]:
#Scree plot: first 10 PCs

pca_gc_10 = PCA(n_components=10) #the first ten PC
pca_gc_10.fit(scaled_data)

gc_pca_10 = pca_gc_10.transform(scaled_data)

per_var = np.round(pca_gc_10.explained_variance_ratio_* 100, decimals=1)
labels = ['PC' + str(x) for x in range(1, len(per_var)+1)]
 
plt.bar(x=range(1,len(per_var)+1), height=per_var, tick_label=labels)
plt.ylabel('Percentage of Explained Variance')
plt.xlabel('Principal Component')
plt.title('Scree Plot (gene exp. and cell viab.)')
plt.show()

In [ ]:
#Plotting the features w.r.t. CP1 and CP2:

plt.figure(figsize=(8,6))
plt.scatter(gc_pca[:,0],gc_pca[:,1])
plt.xlabel('First principle component')
plt.ylabel('Second principle component')

<div id="lr"></div>

## 5. A Logistic Regression Model (neural networks with no hidden layers)

### 5.1. Lambda optimization

https://www.kaggle.com/heitorbaldo/moa-prediction-pca-dnn-r

### 5.2. Predictions for the original data

In [ ]:
#Building the Logistic Regression model (neural network without hidden layers):

def LR_Model(inputsize):
    classifier = Sequential()
    classifier.add(Dense(206, activation="sigmoid", 
                              kernel_regularizer=regularizers.l1(0),
                              input_dim=inputsize))
    
    classifier.compile(optimizer='adam', loss = 'binary_crossentropy', 
                       metrics = ['accuracy'])
    
    return classifier

In [ ]:
#Fitting the LR model (original data):

LR_Model_orig = LR_Model(875)

history_lr_orig = LR_Model_orig.fit(x_train_orig, 
                              y_train_orig,
                              validation_split=0.3,
                              epochs=300, 
                              batch_size=2300,
                              verbose=0,
                              callbacks=[TqdmCallback()])

In [ ]:
#Training / Validation Loss (original data)

trg_loss_orig = history_lr_orig.history['loss']
val_loss_orig = history_lr_orig.history['val_loss']
epochs = range(1, 301)

fig = plt.figure(figsize=(20,8))
ax = fig.add_subplot(1, 2, 1)
plt.plot(epochs, trg_loss_orig, 'b',  linewidth=3, label='Training Loss')
plt.plot(epochs, val_loss_orig, 'r',  linewidth=3, label='Validation Loss')
plt.title("Training / Validation Loss - LR model (original data)")
ax.set_ylabel("Loss")
ax.set_xlabel("Epochs")
ax.set_facecolor("white")
plt.legend(loc='best')
    
plt.tight_layout()
plt.show()

In [ ]:
#Loss for the test partition:

LR_Model_orig.evaluate(x_test_orig, y_test_orig)
print(min(history_lr_orig.history["val_loss"]))
print(history_lr_orig.history["val_loss"][299])

### 5.3. Predictions for the reduced data

In [ ]:
#Fitting the LR model (reducted data - without clt_vehicle):

LR_Model_red = LR_Model(875)

history_lr_red = LR_Model_red.fit(x_train_wv, 
                              y_train_wv,
                              validation_split=0.3,
                              epochs=300, 
                              batch_size=2300,
                              verbose=0,
                              callbacks=[TqdmCallback()])

In [ ]:
#Training / Validation Loss (reduced data)

trg_loss_red = history_lr_red.history['loss']
val_loss_red = history_lr_red.history['val_loss']
epochs = range(1, 301)

fig = plt.figure(figsize=(20,8))
ax = fig.add_subplot(1, 2, 1)
plt.plot(epochs, trg_loss_red, 'k',  linewidth=3, label='Training Loss')
plt.plot(epochs, val_loss_red, 'y',  linewidth=3, label='Validation Loss')
plt.title("Training / Validation Loss - LR model (reduced data)")
ax.set_ylabel("Loss")
ax.set_xlabel("Epochs")
ax.set_facecolor("white")
plt.legend(loc='best')
    
plt.tight_layout()
plt.show()

In [ ]:
#Loss for the test partition

LR_Model_red.evaluate(x_test_wv, y_test_wv)
print(min(history_lr_red.history["val_loss"]))
print(history_lr_red.history["val_loss"][299])

### 5.4. Predictions for the PCA data

In [ ]:
#Fitting the LR model (PCA data):

LR_Model_pca = LR_Model(490)

history_lr_pca = LR_Model_pca.fit(x_train_pca, 
                              y_train_pca,
                              validation_split=0.3,
                              epochs=300, 
                              batch_size=2300,
                              verbose=0,
                              callbacks=[TqdmCallback()])

In [ ]:
#Training / Validation Loss (PCA data)

trg_loss_pca = history_lr_pca.history['loss']
val_loss_pca = history_lr_pca.history['val_loss']
epochs = range(1, 301)

fig = plt.figure(figsize=(20,8))
ax = fig.add_subplot(1, 2, 1)
plt.plot(epochs, trg_loss_pca, 'g',  linewidth=3, label='Training Loss')
plt.plot(epochs, val_loss_pca, 'm',  linewidth=3, label='Validation Loss')
plt.title("Training / Validation Loss - LR model (PCA data)")
ax.set_ylabel("Loss")
ax.set_xlabel("Epochs")
ax.set_facecolor("white")
plt.legend(loc='best')
    
plt.tight_layout()
plt.show()

In [ ]:
#Loss for the test partition

LR_Model_pca.evaluate(x_test_pca, y_test_pca)
print(min(history_lr_pca.history["val_loss"]))
print(history_lr_pca.history["val_loss"][299])

### 5.5. Predictions for the reduced PCA data

In [ ]:
#Fitting the LR model (reduced PCA data):

LR_Model_pca_wv = LR_Model(490)

history_lr_pca_wv = LR_Model_pca.fit(x_train_pca_wv, 
                              y_train_pca_wv,
                              validation_split=0.3,
                              epochs=300, 
                              batch_size=2300,
                              verbose=0,
                              callbacks=[TqdmCallback()])

In [ ]:
#Training / Validation Loss (reduced PCA data)

trg_loss_pca_wv = history_lr_pca_wv.history['loss']
val_loss_pca_wv = history_lr_pca_wv.history['val_loss']
epochs = range(1, 301)

fig = plt.figure(figsize=(20,8))
ax = fig.add_subplot(1, 2, 1)
plt.plot(epochs, trg_loss_pca_wv, 'c',  linewidth=3, label='Training Loss')
plt.plot(epochs, val_loss_pca_wv, 'darkgreen',  linewidth=3, label='Validation Loss')
plt.title("Training / Validation Loss - LR model (reduced PCA data)")
ax.set_ylabel("Loss")
ax.set_xlabel("Epochs")
ax.set_facecolor("white")
plt.legend(loc='best')
    
plt.tight_layout()
plt.show()

In [ ]:
#Loss for the test partition

LR_Model_pca_wv.evaluate(x_test_pca_wv, y_test_pca_wv)
print(min(history_lr_pca_wv.history["val_loss"]))
print(history_lr_pca_wv.history["val_loss"][299])

### 5.6. Comparing the results

In [ ]:
#Validation Loss (original, reduced, PCA, and reduced PCA data)   

fig = plt.figure(figsize=(20, 8))
ax = fig.add_subplot(1, 2, 1)
plt.plot(epochs, val_loss_orig, 'r',  linewidth=3, label='Validation Loss (original)')
plt.plot(epochs, val_loss_red, 'y',  linewidth=3, label='Validation Loss (reduced)')
plt.plot(epochs, val_loss_pca, 'm',  linewidth=3, label='Validation Loss (PCA)')
plt.plot(epochs, val_loss_pca_wv, 'darkgreen',  linewidth=3, label='Validation Loss (reduced PCA)')
plt.title("Validation Loss - LR model (original, reduced, PCA, and reduced PCA data)  ")
ax.set_ylabel("Loss")
ax.set_xlabel("Epochs")
ax.set_facecolor("white")
ax.patch.set_edgecolor('black')
ax.patch.set_linewidth('1')  
plt.legend(loc='best')
    
plt.tight_layout()
plt.show()

<div id="dnn"></div>

## 6. Multi-label classification via Deep Neural Networks

<div id="#hyper"></div>

### 6.1. Hyperparameter optimization (tuning)

In [ ]:
#Hyperparameter optimization (tuning)
#Neural network model: 

'''
def NeuralNet(optimizer, kernel_initializer, activation, neurons, drop_rate):
    classifier = Sequential()
    classifier.add(Dense(units = neurons, activation=activation,
                        kernel_initializer=kernel_initializer, input_dim=875))
    classifier.add(BatchNormalization())
    classifier.add(Dropout(drop_rate))
    classifier.add(Dense(units = neurons, activation=activation,
                        kernel_initializer=kernel_initializer))
    classifier.add(BatchNormalization())
    classifier.add(Dropout(drop_rate))
    classifier.add(Dense(units = neurons, activation=activation,
                        kernel_initializer=kernel_initializer))
    classifier.add(BatchNormalization())
    classifier.add(Dense(206, activation="sigmoid"))
    classifier.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = NeuralNet)
'''

#Grid Search method:

'''
parameters = {'batch_size': [1000, 2300],
              'epochs': [50, 100, 300],
              'optimizer': ['adam', 'adamw'],
              'kernel_initializer': ['random_uniform', 'normal'], 
              'activation': ['relu', 'leaky-relu', 'elu'],
              'neurons': [1048, 512, 256, 128],
              'drop_rate': [0.2, 0.3, 0.4]}

grid_search = GridSearchCV(estimator = classifier, 
                           param_grid = parameters, 
                           scoring = 'accuracy', cv = 5)

grid_search = grid_search.fit(x_train_orig, y_train_orig)

best_param = grid_search.best_params_
best_param
best_acc = grid_search.best_score_
best_acc
'''

In [ ]:
#Hyperparameters were set by the Grid Search method (previous code):

batchsize_hyp = 2300
epochs_hyp = 300
hiddenlayers_hyp = 512
dropout_hyp = 0.3
opt_hyp = 'adam'

### 6.2. Predictions for the original data

In [ ]:
#Building the DNN model:

def DNN_Model(inputsize):
    classifier = Sequential()
    classifier.add(WeightNormalization(Dense(512, activation="relu", input_dim=inputsize)))
    classifier.add(BatchNormalization())
    classifier.add(Dropout(0.3))
    classifier.add(WeightNormalization(Dense(256, activation="relu"))) 
    classifier.add(BatchNormalization())
    classifier.add(WeightNormalization(Dense(128, activation="relu"))) 
    classifier.add(BatchNormalization())
    classifier.add(WeightNormalization(Dense(206, activation="sigmoid"))) 
    
    classifier.compile(optimizer='adam', loss = 'binary_crossentropy', 
                       metrics = ['accuracy'])
    
    return classifier

In [ ]:
#Fitting the DNN model (original data):

DNN_Model_orig = DNN_Model(875)

history_dnn_orig = DNN_Model_orig.fit(x_train_orig, 
                              y_train_orig,
                              validation_split=0.3,
                              epochs=300, 
                              batch_size=2300,
                              verbose=0,
                              callbacks=[TqdmCallback()])

In [ ]:
#Training / Validation Loss (original data)

trg_loss_dnn_orig = history_dnn_orig.history['loss']
val_loss_dnn_orig = history_dnn_orig.history['val_loss']
epochs = range(1, 301)

fig = plt.figure(figsize=(20,8))
ax = fig.add_subplot(1, 2, 1)
plt.plot(epochs, trg_loss_dnn_orig, 'b',  linewidth=3, label='Training Loss')
plt.plot(epochs, val_loss_dnn_orig, 'r',  linewidth=3, label='Validation Loss')
plt.title("Training / Validation Loss - DNN model (original data)")
ax.set_ylabel("Loss")
ax.set_xlabel("Epochs")
ax.set_facecolor("white")
plt.legend(loc='best')
    
plt.tight_layout()
plt.show()

In [ ]:
#Loss for the test partition

DNN_Model_orig.evaluate(x_test_orig, y_test_orig)
print(min(history_dnn_orig.history["val_loss"]))
print(history_dnn_orig.history["val_loss"][299])

<div id="#train-red"></div>

### 6.3. Predictions for the reduced data

In [ ]:
#Fitting the DNN model (reducted data - without clt_vehicle):

DNN_Model_red = DNN_Model(875)

history_dnn_red = DNN_Model_red.fit(x_train_wv, 
                              y_train_wv,
                              validation_split=0.3,
                              epochs=300, 
                              batch_size=2300,
                              verbose=0,
                              callbacks=[TqdmCallback()])

In [ ]:
#Training / Validation Loss (reduced data)

trg_loss_dnn_red = history_dnn_red.history['loss']
val_loss_dnn_red = history_dnn_red.history['val_loss']
epochs = range(1, 301)

fig = plt.figure(figsize=(20,8))
ax = fig.add_subplot(1, 2, 1)
plt.plot(epochs, trg_loss_dnn_red, 'k',  linewidth=3, label='Training Loss')
plt.plot(epochs, val_loss_dnn_red, 'y',  linewidth=3, label='Validation Loss')
plt.title("Training / Validation Loss - DNN model (reduced data)")
ax.set_ylabel("Loss")
ax.set_xlabel("Epochs")
ax.set_facecolor("white")
plt.legend(loc='best')
    
plt.tight_layout()
plt.show()

In [ ]:
#Loss for the test partition

DNN_Model_red.evaluate(x_test_wv, y_test_wv)
print(min(history_dnn_red.history["val_loss"]))
print(history_dnn_red.history["val_loss"][299])

<div id="#train-pca"></div>

### 6.4. Predictions for the PCA data

In [ ]:
#Fitting the DNN model (PCA data):

DNN_Model_pca = DNN_Model(490)

history_dnn_pca = DNN_Model_pca.fit(x_train_pca, 
                              y_train_pca,
                              validation_split=0.3,
                              epochs=300, 
                              batch_size=2300,
                              verbose=0,
                              callbacks=[TqdmCallback()])

In [ ]:
#Training / Validation Loss (PCA data)

trg_loss_dnn_pca = history_dnn_pca.history['loss']
val_loss_dnn_pca = history_dnn_pca.history['val_loss']
epochs = range(1, 301)

fig = plt.figure(figsize=(20,8))
ax = fig.add_subplot(1, 2, 1)
plt.plot(epochs, trg_loss_dnn_pca, 'g',  linewidth=3, label='Training Loss')
plt.plot(epochs, val_loss_dnn_pca, 'm',  linewidth=3, label='Validation Loss')
plt.title("Training / Validation Loss - DNN model (PCA data)")
ax.set_ylabel("Loss")
ax.set_xlabel("Epochs")
ax.set_facecolor("white")
plt.legend(loc='best')
    
plt.tight_layout()
plt.show()

In [ ]:
#Loss for the test partition

DNN_Model_pca.evaluate(x_test_pca, y_test_pca)
print(min(history_dnn_pca.history["val_loss"]))
print(history_dnn_pca.history["val_loss"][299])

<div id="#train-pca_wv"></div>

### 6.5. Predictions for the reduced PCA data

In [ ]:
#Fitting the DNN model (reduced PCA data):

DNN_Model_pca_wv = DNN_Model(875)

history_dnn_pca_wv = DNN_Model_pca_wv.fit(x_train_pca_wv, 
                              y_train_pca_wv,
                              validation_split=0.3,
                              epochs=300, 
                              batch_size=2300,
                              verbose=0,
                              callbacks=[TqdmCallback()])

In [ ]:
#Training / Validation Loss (original data)

trg_loss_dnn_pca_wv = history_dnn_pca_wv.history['loss']
val_loss_dnn_pca_wv = history_dnn_pca_wv.history['val_loss']
epochs = range(1, 301)

fig = plt.figure(figsize=(20,8))
ax = fig.add_subplot(1, 2, 1)
plt.plot(epochs, trg_loss_dnn_pca_wv, 'c',  linewidth=3, label='Training Loss')
plt.plot(epochs, val_loss_dnn_pca_wv, 'darkgreen',  linewidth=3, label='Validation Loss')
plt.title("Training / Validation Loss - DNN model (original data)")
ax.set_ylabel("Loss")
ax.set_xlabel("Epochs")
ax.set_facecolor("white")
plt.legend(loc='best')
    
plt.tight_layout()
plt.show()

In [ ]:
#Loss for the test partition

DNN_Model_pca_wv.evaluate(x_test_pca_wv, y_test_pca_wv)
print(min(history_dnn_pca_wv.history["val_loss"]))
print(history_dnn_pca_wv.history["val_loss"][299])

<div id="#comp"></div>

### 6.6. Comparing the results

In [ ]:
#Validation Loss (original, reduced, PCA, and reduced PCA data)   

fig = plt.figure(figsize=(20, 8))
ax = fig.add_subplot(1, 2, 1)
plt.plot(epochs, val_loss_dnn_orig, 'r',  linewidth=3, label='Validation Loss (original)')
plt.plot(epochs, val_loss_dnn_red, 'y',  linewidth=3, label='Validation Loss (reduced)')
plt.plot(epochs, val_loss_dnn_pca, 'm',  linewidth=3, label='Validation Loss (PCA)')
plt.plot(epochs, val_loss_dnn_pca_wv, 'darkgreen',  linewidth=3, label='Validation Loss (reduced PCA)')
plt.title("Validation Loss - DNN model (original, reduced, PCA, and reduced PCA data)  ")
ax.set_ylabel("Loss")
ax.set_xlabel("Epochs")
ax.set_facecolor("white")
plt.legend(loc='best')
    
plt.tight_layout()
plt.show()

### 6.7. Comparing the models

In [ ]:
#Validation Loss - DNN vs LR model (origina and PCA data)   

fig = plt.figure(figsize=(20, 8))
ax = fig.add_subplot(1, 2, 1)
plt.plot(epochs, val_loss_orig, 'r',  linewidth=3, label='Validation Loss (LR - original)')
plt.plot(epochs, val_loss_pca, 'y',  linewidth=3, label='Validation Loss (LR - PCA)')
plt.plot(epochs, val_loss_dnn_orig, 'm',  linewidth=3, label='Validation Loss (DNN - original)')
plt.plot(epochs, val_loss_dnn_pca, 'darkgreen',  linewidth=3, label='Validation Loss (DNN - PCA)')
plt.title("Validation Loss - DNN vs LR model (original and PCA data)")
ax.set_ylabel("Loss")
ax.set_xlabel("Epochs")
ax.set_facecolor("white")
plt.legend(loc='best')
    
plt.tight_layout()
plt.show()

<div id="submit"></div>

## 7. Submission

In [ ]:
sample_submit = pd.read_csv('/kaggle/input/lish-moa/sample_submission.csv')
sample_submit.head()

In [ ]:
submit_pred = DNN_Model_orig.predict(test)

In [ ]:
sample_submit.iloc[:,1:] = submit_pred
sample_submit.head()

In [ ]:
sample_submit.to_csv("submission.csv", index=False, header=True) 